# Train-Test Split

Step 1 - Read merged_data_analysis.xlsx as tables, remove variables with IS_KEEP = FALSE  
Step 2 - Merge data with diabetes_medication output  
Step 3 - Split data into train-test  
Step 4 - Perform distribution test, Kolgomorov-Smirnov for continuous, Chi-square for categorical  